In [1]:
import pandas as pd

# docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))['data']
# print(len(docs))
papers = pd.read_csv('./data_face/data_groups_d23_ptit.csv')
print(papers.columns)
# papers.rename(columns={'body': 'paper_text'}, inplace=True)
# convert type of column
papers['text'].astype(str)
# Print head
print(papers.head())
print(papers.shape)

Index(['post_id', 'text', 'timestamp', 'time', 'user_id'], dtype='object')
           post_id                                               text  \
0  640719428178910  🔥CÁC KÊNH YOUTUBE HỌC CÁC MÔN🔥\nVào từng ảnh đ...   
1  659246332992886                          ae malibu thua nnn chưa ạ   
2  659210466329806  đ*t cụ mấy thằng ptit ở malibu giờ đi ngủ ngậm...   
3  659176019666584      kiếm mập mờ ở malibu gấp, lạnh lắm rồiiiiiii🥶   
4  659172869666899  Long bào đã kịp về để Tết này còn flex với họ ...   

      timestamp                 time           user_id  
0  1.696861e+09  2023-10-09 21:10:58   100042024003239  
1  1.699863e+09  2023-11-13 15:06:26   284734091022087  
2  1.699857e+09  2023-11-13 13:26:16   306021115550368  
3  1.699850e+09  2023-11-13 11:41:09  1482440865923204  
4  1.699850e+09  2023-11-13 11:31:56   100037648460053  
(3186, 5)


In [2]:
import numpy as np
# duplicated
print(papers.duplicated().sum())
papers.drop_duplicates(inplace=True)

# drop nan
papers.dropna(axis=0, inplace=True, how="any")
# drop inf
papers.replace([np.inf, -np.inf], np.nan, inplace=True)
papers.replace('', np.nan, inplace=True)
papers.dropna(inplace=True,how='any',axis=0)
# Remove the columns
try:
    papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)
except:
    pass

# Print out the first rows of papers
print(papers.head())
print(papers.info())
papers.reset_index(inplace=True, drop=True)

0
           post_id                                               text  \
0  640719428178910  🔥CÁC KÊNH YOUTUBE HỌC CÁC MÔN🔥\nVào từng ảnh đ...   
1  659246332992886                          ae malibu thua nnn chưa ạ   
2  659210466329806  đ*t cụ mấy thằng ptit ở malibu giờ đi ngủ ngậm...   
3  659176019666584      kiếm mập mờ ở malibu gấp, lạnh lắm rồiiiiiii🥶   
4  659172869666899  Long bào đã kịp về để Tết này còn flex với họ ...   

      timestamp                 time           user_id  
0  1.696861e+09  2023-10-09 21:10:58   100042024003239  
1  1.699863e+09  2023-11-13 15:06:26   284734091022087  
2  1.699857e+09  2023-11-13 13:26:16   306021115550368  
3  1.699850e+09  2023-11-13 11:41:09  1482440865923204  
4  1.699850e+09  2023-11-13 11:31:56   100037648460053  
<class 'pandas.core.frame.DataFrame'>
Index: 3165 entries, 0 to 3185
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   post_id    3165 non-null   int6

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from underthesea import word_tokenize, sent_tokenize
from underthesea import ner
import torch

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

def ner_results(text) -> dict[str, list[str]]: # dict[str, list[str]]: key là tên thực thể, value là list các từ thuộc thực thể đó
    sentences = sent_tokenize(text)
    # print(sentences)
    sentences = [s for s in sentences if len(s) > 0]

    list_ner = []
    results_sentences = []

    for s in sentences:
        if len(s) > 0:
            tensor_input = torch.tensor([tokenizer.encode(s)])
            # print(tensor_input.shape)
            if(tensor_input.shape[1] < 512):
                # print(preprocess(s))
                # print(nlp(s))
                if len(nlp(s)) > 0:
                    word = ""
                    original_word = ""
                    # s = preprocess(s)
                    for i in nlp(s):
                        if 'B-' in i['entity']:
                            word = i['word']
                            original_word = i['word']
                        elif 'I-' in i['entity']:
                            original_word += " " + i['word']
                            if i['word'] != "-":
                                word += " " + i['word']
                        elif 'E-' in i['entity']:
                            word += " " + i['word']
                            original_word += " " + i['word']
                            list_ner.append({'entity': i['entity'].split('-')[1], 'word': word})
                            word = ""
                            original_word = ""
                        elif 'S-' in i['entity']:
                            list_ner.append({'entity': i['entity'].split('-')[1], 'word': i['word']})
                            word = ""
                            original_word = ""
                    if word != "":
                        list_ner.append({'entity': i['entity'].split('-')[1], 'word': word})
                        # print(word)
                        s = s.replace(original_word, "_".join(word.split(" ")).lower())
                        # print(s)
                        results_sentences.append(s)
                        word = ""
                        original_word = ""
                #
                list_ner += nlp(s)
                # print(nlp(preprocess(s)))
                # print("=========================================")
            else:
                print("sentence too long")

    # print(list_ner)
    dict_ner = {}
    for i in list_ner:
        if i['entity'] in dict_ner:
            dict_ner[i['entity']].append(i['word'])
        else:
            dict_ner[i['entity']] = [i['word']]

    results = {}
    for key, value in dict_ner.items():
        if '-' not in key:
            # print(key + ": " + str(value))
            results[key] = value
    
    print(results_sentences)
    return results, " ".join(results_sentences)

ner_results_location_ = []
ner_results_person_ = []
ner_results_organization_ = []
doc = []
for i in range(len(papers['text'])):
    results, doc_ = ner_results(papers['text'][i])
    dict_ner = results
    doc.append(doc_)
    print(i)
    if 'LOCATION' not in dict_ner:
        dict_ner['LOCATION'] = []
    if 'PERSON' not in dict_ner:
        dict_ner['PERSON'] = []
    if 'ORGANIZATION' not in dict_ner:
        dict_ner['ORGANIZATION'] = []    
    ner_results_location_.append(dict_ner['LOCATION'])
    ner_results_person_.append(dict_ner['PERSON'])
    ner_results_organization_.append(dict_ner['ORGANIZATION'])
    # print(dict_ner)
    print(doc_)
    
papers['location'] = ner_results_location_
papers['person'] = ner_results_person_
papers['organization'] = ner_results_organization_
papers['doc_ner'] = doc
papers.to_csv('./data_face/data_group_ner.csv', index=False)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]
0

[]
1

['đ*t cụ mấy thằng ptit ở malibu giờ đi ngủ ngậm mồm vào tắt âm đê ko ngủ để ngkhac ngủ, lớn rồi ý thức lên']
2
đ*t cụ mấy thằng ptit ở malibu giờ đi ngủ ngậm mồm vào tắt âm đê ko ngủ để ngkhac ngủ, lớn rồi ý thức lên
['kiếm mập mờ ở malibu gấp, lạnh lắm rồiiiiiii🥶']
3
kiếm mập mờ ở malibu gấp, lạnh lắm rồiiiiiii🥶
[]
4

[]
5

['Khong có em cũng khong sao🔥\n#malibu ngày đông 🥶']
6
Khong có em cũng khong sao🔥
#malibu ngày đông 🥶
[]
7

[]
8

[]
9

[]
10

['Có ai mê mùi nước giặt ở thiên_thai như em khum ạ 🥰 ultr xin info nc giặt về còn mua\nĐáng để thử !']
11
Có ai mê mùi nước giặt ở thiên_thai như em khum ạ 🥰 ultr xin info nc giặt về còn mua
Đáng để thử !
[]
12

[]
13

[]
14

['v ơi tớ lạnh quá 🥶']
15
v ơi tớ lạnh quá 🥶
[]
16

[]
17

[]
18

[]
19

['Mấy bạn ở c8 ra ngoài hành lang hát đồng ca kh ạ']
20
Mấy bạn ở c8 ra ngoài hành lang hát đồng ca kh ạ
[]
21

['Xin in4 3 bạn nữ tầng 1 toà c7 đã cho tk bạn mình đi nhờ ô , nó muốn cảm ơn \U0001faf6']
22
Xin in4 3 bạn nữ tầng 1 to

KeyboardInterrupt: 